In [32]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

from res_ind_lib import average_over_rp

import glob

from progress_reporter import progress_reporter

%matplotlib inline
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy
def wp(data, wt, percentiles,cum=False): 
	"""Compute weighted percentiles. 
	If the weights are equal, this is the same as normal percentiles. 
	Elements of the C{data} and C{wt} arrays correspond to 
	each other and must have equal length (unless C{wt} is C{None}). 

	@param data: The data. 
	@type data: A L{numpy.ndarray} array or a C{list} of numbers. 
	@param wt: How important is a given piece of data. 
	@type wt: C{None} or a L{numpy.ndarray} array or a C{list} of numbers. 
		 All the weights must be non-negative and the sum must be 
		 greater than zero. 
	@param percentiles: what percentiles to use.  (Not really percentiles, 
		 as the range is 0-1 rather than 0-100.) 
	@type percentiles: a C{list} of numbers between 0 and 1. 
	@rtype: [ C{float}, ... ] 
	@return: the weighted percentiles of the data. 
	"""
	assert numpy.greater_equal(percentiles, 0.0).all(), "Percentiles less than zero" 
	assert numpy.less_equal(percentiles, 1.0).all(), "Percentiles greater than one" 
	data = numpy.asarray(data) 
	# data = numpy.reshape(data,(len(data)))
	assert len(data.shape) == 1 
	if wt is None: 
		 wt = numpy.ones(data.shape, numpy.float) 
	else: 
		 wt = numpy.asarray(wt, numpy.float) 
		 # wt = numpy.reshape(wt,(len(wt)))
		 assert wt.shape == data.shape 
		 assert numpy.greater_equal(wt, 0.0).all(), "Not all weights are non-negative." 
	i = numpy.argsort(data) 
	sd = numpy.take(data, i, axis=0)
	sw = numpy.take(wt, i, axis=0) 
	aw = numpy.add.accumulate(sw) 
	if not aw[-1] > 0: 
		 raise ValueError("Nonpositive weight sum" )
	w = (aw)/aw[-1] 
	spots = numpy.searchsorted(w, percentiles) 
	if cum:
		sd = numpy.add.accumulate(numpy.multiply(sd,sw))
	f = interp1d(w,sd)
	return f(percentiles)
	

In [81]:
from tempfile import mkstemp, SpooledTemporaryFile
from shutil import move
from os import remove, close
from  io import StringIO

def res_to_csv(file_path, new_path=None):
    #Create temp file
    new_file =StringIO()
    
    #actually replaces text line by line
    
    data = False
    

    with open(file_path) as old_file:
        for line in old_file:
            if line.startswith("Temporalidad"): #begining of data
                data=True
                colindex = line.find("Frecuencia")

            if line.startswith("Curva"):
                break

            if data:   
                try:
                    new_file.write(line[colindex:].replace("NeuN"," 0 "))
                except IndexError:
                    print("one line omited")    
    

    
    #closes the temporary file
    return new_file


In [4]:
return_periods = np.array( [ 0.1,.5,1,1.5, 2,3, 5, 10, 20, 50, 100, 250, 500, 1000, 1500, 2000, 4000, 5000, 30e3])



In [21]:
iso2bank = pd.read_csv("inputs/iso3_to_wb_name.csv",index_col="iso3", squeeze=True)

In [23]:

res_file_list = glob.glob("D:/events/*/*.res")





In [43]:
PML_guessed = pd.DataFrame()

In [31]:
myfile = res_file_list[0]

In [84]:

for myfile in res_file_list:

    #parse filename
    path, name = glob.os.path.split(myfile)
    hazard = path.split("\\")[-1]
    try:
        country_name = iso2bank[name.split("_")[1]]
    except KeyError:
        print("ignored " +myfile)
        continue
    
    
    if ((hazard,country_name) in PML_guessed):
        continue
    
    progress_reporter((hazard,country_name))

    try:
        #open file and parse it to memory file
        memory_file = res_to_csv(myfile)
        #gets back to begining or IO
        memory_file.seek(0)

        #read data
        data = pd.read_csv(memory_file,sep=" *", engine="python",usecols=["Frecuencia","EP"])
        data= data.sort_values(by="EP", ascending=False)[["EP","Frecuencia"]]

        data["rp"]=1/data.Frecuencia.cumsum()
        data.head()

        #interpolates RP
        series = pd.DataFrame(interp1d(data.rp,data.EP, bounds_error=False)(return_periods), 
                              index=pd.Index(return_periods,name="rp"))

        #scales fore same average capital losses
        series *= (data.EP*data.Frecuencia).sum()/ average_over_rp(series.stack()).squeeze()

        PML_guessed[(hazard,country_name)]=series.squeeze()
    except:
        pass


Currently working on:  ('tsunami', 'Saudi Arabia')
ignored D:/events\wind\africa_MYT_Wd_Total.res
ignored D:/events\wind\africa_REU_Wd_Total.res
ignored D:/events\wind\lac_GLP_Wd_Total.res
ignored D:/events\wind\lac_MSR_Wd_Total.res
ignored D:/events\wind\lac_MTQ_Wd_Total.res


In [45]:
PML_guessed.head()

,"(earthquake, Angola)","(earthquake, Burundi)","(earthquake, Benin)","(earthquake, Botswana)","(earthquake, Central African Republic)","(earthquake, Cameroon)","(earthquake, Congo, Dem. Rep.)","(earthquake, Congo, Rep.)","(earthquake, Comoros)","(earthquake, Djibouti)",...,"(wind, Puerto Rico)","(wind, Turks and Caicos Islands)","(wind, Trinidad and Tobago)","(wind, St. Vincent and the Grenadines)","(wind, Venezuela, RB)","(wind, Virgin Islands, British)","(wind, Virgin Islands (U.S.))","(wind, Canada)","(wind, United States)","(wind, Oman)"
rp,,,,,,,,,,,,,,,,,,,,,
0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN
0.5,NaN,0.117194,NaN,NaN,NaN,NaN,0.263547,NaN,NaN,0.041216,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,8.791047,NaN
1.0,NaN,0.512858,NaN,10.605309,NaN,NaN,0.706891,NaN,NaN,0.147379,...,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,1398.196269,0.0
1.5,0.007520,0.833187,NaN,15.715881,0.000449,NaN,1.173001,NaN,0.005841,0.258941,...,0.001545,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.133757,4983.512957,0.0
2.0,0.027204,1.167576,NaN,18.866049,0.000848,NaN,1.495127,NaN,0.036452,0.379879,...,4.144791,0.010392,0.0,0.0,0.0,0.003585,0.054135,1.205804,8821.118707,0.0


In [85]:
df=PML_guessed.copy()
df.columns=pd.MultiIndex.from_tuples(PML_guessed.columns)
df=df.stack().stack()
df.index.names=["rp","country","hazard"]
df.swaplevel(0,1).swaplevel(1,2).sort_index().to_csv("intermediate/capital_losses_from_GAR_events.csv", header=True)

In [92]:
average_over_rp(df).sum()

0    284144.269285
dtype: float64

In [82]:
sum=0

for myfile in res_file_list:

    #parse filename
    path, name = glob.os.path.split(myfile)
    hazard = path.split("\\")[-1]
    
    if hazard !="surge":
        continue

    try:
        #open file and parse it to memory file
        memory_file = res_to_csv(myfile)
        #gets back to begining or IO
        memory_file.seek(0)

        #read data
        data = pd.read_csv(memory_file,sep=" *", engine="python",usecols=["Frecuencia","EP"])
        data= data.sort_values(by="EP", ascending=False)[["EP","Frecuencia"]]   

        #scales fore same average capital losses
        sum+= (data.EP*data.Frecuencia).sum()
    except:
        print("passing "+myfile)




In [90]:
49788 + 247608

297396

In [83]:
suma

49788.27928993283

In [73]:
sum

247608.99403434165

In [87]:
myfile = "D:/events\wind\africa_REU_Wd_Total.res"

In [88]:
path, name = glob.os.path.split(myfile)
hazard = path.split("\\")[-1]
try:
    country_name = iso2bank[name.split("_")[1]]
except KeyError:
    print("ignored " +myfile)


ignored D:/events\windfrica_REU_Wd_Total.res


In [91]:
memory_file = res_to_csv(myfile)
#gets back to begining or IO
memory_file.seek(0)

#read data
data = pd.read_csv(memory_file,sep=" *", engine="python",usecols=["Frecuencia","EP"])
data= data.sort_values(by="EP", ascending=False)[["EP","Frecuencia"]]

data["rp"]=1/data.Frecuencia.cumsum()
data.head()

#interpolates RP
series = pd.DataFrame(interp1d(data.rp,data.EP, bounds_error=False)(return_periods), 
                      index=pd.Index(return_periods,name="rp"))

#scales fore same average capital losses
series *= (data.EP*data.Frecuencia).sum()/ average_over_rp(series.stack()).squeeze()

PML_guessed[(hazard,country_name)]=series.squeeze()

OSError: [Errno 22] Invalid argument: 'D:/events\\wind\x07frica_REU_Wd_Total.res'